# We're Gonna Need a Bigger Bot

Let's map all the bits and pieces to a meatier example, to see how hello-ltr's abstractions make it easier to play with LTR via ipynbs

Genome-tags is a crowdsourced movie tagging resource project. Each movie is assigned from 0-1 how close a movie matches a tag. Luckily the tags look remarkably like search queries (ie `star trek` or `berlin`). We derrived judgments from the genome-tags data, and use them to experiment with search.

BUT

While some tags are straight forward (`Star Trek`) others are much tougher (`boxing` or `french art movie`) where its unlikely any text has a match. 

We're not going to solve those problems here, but we give this to you as a sandbox to apply your skills after the class to see how close you can get to approximating the genome tags data. 


### Clients 

While syntaxes differ, the LTR process is nearly identical between Solr and Elastic. So you can repeat a lot of the labs with Solr (some have been already translated)

But we'll stick with Elastic

In [1]:
from ltr.client import OpenSearchClient
client = OpenSearchClient()

http://localhost:9201/_ltr; <OpenSearch([{'host': 'localhost', 'port': 9201}])>


### Redownload corpus & judgments if you need to

In [2]:
from ltr import download
corpus='http://es-learn-to-rank.labs.o19s.com/tmdb.json'
judgments='http://es-learn-to-rank.labs.o19s.com/genome_judgments.txt'

download([corpus, judgments], dest='data/');

data/tmdb.json already exists
GET http://es-learn-to-rank.labs.o19s.com/genome_judgments.txt


### Reindex if you need to

In [3]:
from ltr.index import rebuild
from ltr.helpers.movies import indexable_movies

movies=indexable_movies(movies='data/tmdb.json')
rebuild(client, index='tmdb', doc_src=movies)

Index tmdb already exists. Use `force = True` to delete and recreate


###  Feature Sets in ipynb

You played with creating a feature set in the last lab, see the same process repeated here.

Learning to rank requires creating feature set. Each feature has a name like `title_bm25` and as part of a list an ordinal `title_bm25` is the 0th item. Confusingly, Ranklib uses 1-based feature numbering, so feature 0 in this list corresponds to feature 1 in Ranklib training file, that we'll see soon.

Notice also:

- Each feature is a templated query with `{{keywords}}` parameter, that is passed at query time
- We've added a `validation` block, which will run these queries with the specified parameters and index and return any query errors

In [4]:
client.reset_ltr(index='tmdb')

config = {
    "featureset": {
        "features": [
            {
                "name": "title_bm25",
                "params": ["keywords"],
                "template": {
                    "match": {
                        "title": "{{keywords}}"
                    } 
                }
            },
            {
                "name": "overview_bm25",
                "params": ["keywords"],
                "template": {
                        "match": {
                            "overview": {
                                "query": "{{keywords}}",
                            }
                        }

                }
            }
        ]
    },
    "validation": {
              "index": "tmdb",
              "params": {
                  "keywords": "rambo"
              }
    
           }
}

client.create_featureset(index='tmdb', name='genome', ftr_config=config)

Removed Default LTR feature store [Status: 200]
Initialize Default LTR feature store [Status: 200]
Create genome feature set [Status: 201]


### Logging Queries

Logging is one of the more complex operations from an engineering perspective. 

The same query you ran manually when reviewing the slides is rerun here for every query in the source judgment list `judgmentInFile` with some batching when needed.

In [5]:
from ltr.log import FeatureLogger
from ltr.judgments import judgments_open
from itertools import groupby

ftr_logger=FeatureLogger(client, index='tmdb', feature_set='genome')
with judgments_open('data/genome_judgments.txt') as judgment_list:
    for qid, query_judgments in groupby(judgment_list, key=lambda j: j.qid):
        ftr_logger.log_for_qid(judgments=query_judgments, 
                               qid=qid,
                               keywords=judgment_list.keywords(qid))

Recognizing 1128 queries


### Training

Here's where we train the model, under the hood this executes Ranklib just as you ran during the training exercises.

Notice here we're optimizing for NDCG@10


In [6]:
from ltr.ranklib import train
trainResponse = train(client,
                 training_set=ftr_logger.logged,
                 metric2t='NDCG@10',
                 featureSet='genome',
                 index='tmdb',
                 modelName='genome')

/var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar already exists
Running java -jar /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar -ranker 6 -shrinkage 0.1 -metric2t NDCG@10 -tree 50 -bag 1 -leaf 10 -frate 1.0 -srate 1.0 -train /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/training.txt -save data/genome_model.txt 
Delete model genome: 404
Created Model genome [Status: 201]
Model saved


Now that training is done, we can output some statistics about the model, including the training metrics. In future units we'll get more into what this looks like.

Notice the training NDCG isn't that great. When originally run, it was only 0.5885. So pretty far off of the genome data. One challenge of Learning to Rank (and Relevance in general) is trying to figure out the features that can close the gap. 

In [7]:
print("Impact of each feature on the model")
trainLog = trainResponse.trainingLogs[0]
for ftrId, impact in trainLog.impacts.items():
    print("{} - {}".format(client.get_feature_name(config, ftrId), impact))
    
print("trainLog Metric %s" % trainLog.metric())

Impact of each feature on the model
title_bm25 - 637582.3838513177
overview_bm25 - 558539.450163617
trainLog Metric 0.6801


### Search with our model

Here we're going to search using the `genome` model. You can see the LTR query being output (sent to Elasticsearch). You're encouraged to run that directly against Elasticsearch if you like.

Please note, this isn't rescoring. And that's fine for our purposes of directly evaluating the model, in real life you really should run a rescore query.

In [8]:
from ltr import search
search(client, "batman", modelName='genome')

{"size": 5, "query": {"sltr": {"params": {"keywords": "batman", "keywordsList": ["batman"]}, "model": "genome"}}}
{'size': 5, 'query': {'sltr': {'params': {'keywords': 'batman', 'keywordsList': ['batman']}, 'model': 'genome'}}}
Batman: Under the Red Hood 
4.059837 
2010 
['Action', 'Animation'] 
Batman faces his ultimate challenge as the mysterious Red Hood takes Gotham City by firestorm. One part vigilante, one part criminal kingpin, Red Hood begins cleaning up Gotham with the efficiency of Batman, but without following the same ethical code. 
---------------------------------------
Batman: The Dark Knight Returns, Part 2 
4.0364103 
2013 
['Animation', 'Action'] 
Batman has stopped the reign of terror that The Mutants had cast upon his city.  Now an old foe wants a reunion and the government wants The Man of Steel to put a stop to Batman. 
---------------------------------------
Batman Returns 
4.0364103 
1992 
['Action', 'Crime', 'Fantasy', 'Science Fiction', 'Thriller'] 
Having def